This code is used to output results for evaluation.

In [1]:
import torch
import os
from Model import Model
from torchvision import transforms
from torch.autograd import Variable
from torch.nn import functional as F
import cv2
import numpy as np
from PIL import Image
from utils import find_central_point, sum_filter, cross_filter, surf, returnHeatmap
from utils import find_corner
classes = np.array([
        'aeroplane',
        'bicycle',
        'bird',
        'boat',
        'bottle',
        'bus',
        'car',
        'cat',
        'chair',
        'cow',
        'diningtable',
        'dog',
        'horse',
        'motorbike',
        'person',
        'pottedplant',
        'sheep',
        'sofa',
        'train',
        'tvmonitor',
    ])
test_path = './data/VOCdevkit/VOC2007/JPEGImages'
det_path = 'eval/2'

In [2]:
def CAM(input):
    net = Model()
    net.load_state_dict(torch.load('weights/epoch204densenet.pth'))
    net.eval()
    
    features_blosbs = []
    def hook_feature(module, input, output):
        features_blosbs.append(output.data.cpu().numpy())
        
    net._modules.get('features').register_forward_hook(hook_feature)
    
    params = list(net.parameters())
    weight_softmax = np.squeeze(params[-2].data.numpy())
    
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])
    preprocess = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ])
    
    img_pil = Image.open(input)

    img_tensor = preprocess(img_pil)
    img_variable = Variable(img_tensor.unsqueeze(0))
    logit = net(img_variable)
    
    h_x = F.softmax(logit, dim=1).data.squeeze()
    probs, idx = h_x.sort(0, True)
    probs = probs.numpy()
    idx = idx.numpy()
    
    img = cv2.imread(input)
    height, width, _ = img.shape 
    
    for i in range(20):
        print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))
        heatmap = returnHeatmap(features_blosbs[0], weight_softmax, [idx[i]])
        resized_heat_map = cv2.resize(heatmap[0],(width, height)) 
        sum_heat_map, maxi_loc = sum_filter(resized_heat_map)
        #cross filter
        #accumulated_heat_map, maxi_loc = sum_filter(resized_heat_map)
        max_x, max_y = maxi_loc
        
        min, max = find_corner(sum_heat_map, 130)
        min[0], min[1] = min[1], min[0]
        max[0], max[1] = max[1], max[0]
        print(min, max)
        
        #surf
        """min, max = find_corner(resized_heat_map, 130)
        min[0], min[1] = min[1], min[0]
        max[0], max[1] = max[1], max[0]
        print('corner',min, max)
        
        #save croped image
        croped = img[min[1]:max[1], min[0]:max[0]]
        bbox_loc, kp = surf(croped)
        left_top = [bbox_loc[0] + min[0], bbox_loc[1] + min[1]]
        right_bottom = [bbox_loc[2] + min[0], bbox_loc[3] + min[1]]
        min = left_top
        max = right_bottom"""
        
        
        with open(det_path + '/%s.txt' %classes[idx[i]], 'a') as f:
            output = os.path.basename(input) + ' %.4f %d %d %d %d\n' %(probs[i], min[0], min[1], max[0], max[1])
            f.write(output)

In [3]:
if __name__ == '__main__':
    img_names = os.listdir(test_path)
    img_names.sort()

    #img_names = img_names[4740:]
    print('start')
    for img_name in img_names:
        print(img_name)
        CAM(os.path.join(test_path, img_name))

start
000001.jpg
0.850 -> dog
corner [0, 188] [179, 416]
49 32 166 186
0.111 -> person
corner [0, 0] [352, 499]
21 20 337 484
0.020 -> cat
corner [0, 133] [352, 499]
21 11 337 351
0.007 -> chair
corner [0, 0] [352, 499]
21 20 337 484
0.003 -> bottle
corner [0, 0] [352, 499]
21 20 337 484
0.002 -> sheep
corner [0, 0] [352, 499]
21 20 337 484
0.002 -> horse
corner [0, 0] [352, 499]
21 20 337 484
0.002 -> car
corner [0, 0] [352, 499]
21 20 337 484
0.001 -> tvmonitor
corner [0, 0] [352, 499]
21 20 337 484
0.001 -> pottedplant
corner [0, 0] [352, 499]
21 20 337 484
0.001 -> sofa
corner [0, 0] [352, 499]
21 20 337 484
0.000 -> bird
corner [0, 0] [352, 499]
21 20 337 484
0.000 -> diningtable
corner [0, 0] [352, 499]
21 20 337 484
0.000 -> bicycle
corner [0, 0] [352, 499]
21 20 337 484
0.000 -> cow
corner [0, 0] [352, 499]
21 20 337 484
0.000 -> bus
corner [0, 0] [352, 499]
21 20 337 484
0.000 -> motorbike
corner [0, 0] [352, 499]
21 20 337 484
0.000 -> aeroplane
corner [0, 0] [352, 499]
21 20

KeyboardInterrupt: 